In [1]:
import pandas as pd
pd.set_option('display.width', 1000)  # Set the display width to a larger value
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import yfinance as yf
import vectorbt as vbt
from datetime import datetime
from datetime import timedelta as td
from tqdm import tqdm  # Visualize loop progress
from sklearn.linear_model import LinearRegression
import pytz


In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [3]:
# clean up your libs and all this shit

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
vbt.settings.set_theme('dark')
# vbt.settings.plotting.aut_rangebreaks = True

In [5]:
import requests
import json
import os
import datetime

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

| Gap-Up Screener |
|--|
TradingView Data (import)

In [151]:
# today's date string for file naming automation
# today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
# today_date_str = '_2024-06-11.csv' # this is a near perfect outcome example replay day
# today_date_str = '_2024-06-26.csv' # test data sprouts farmers market should be first
today_date_str = '_2024_07-11.csv'


print(today_date_str)

_2024_07-11.csv


In [8]:
# concat base file name with today's date string
# filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
filename = "trading_view_raw_data/tv_screen_gap-up_2024-07-10_612.csv"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
trading_view_raw_df.head(5)

1244 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Relative Volume at Time,Relative Volume 1 day,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 week,Relative Volume 1 month,Highest high 1 month,Highest high 1 month - Currency,Highest high 3 months,Highest high 3 months - Currency,Highest high 6 months,Highest high 6 months - Currency,Highest high 52 weeks,Highest high 52 weeks - Currency,Highest high All Time,Highest high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes
0,AAPL,Apple Inc.,Telecommunications equipment,Electronic technology,NASDAQ,"S&P 500, NASDAQ 100, Dow Jones Industrial Average, NASDAQ Composite, S&P 500 Information Technology, S&P 100, Russell 3000, Dow Jones Composite Average, NASDAQ-100 Technology Sector, Russell 1000, NASDAQ Computer, S&P 500 ESG",3.506598e+12,USD,228.68,USD,229.11,USD,0.7600,USD,0.332342,0.188036,1.531993e+10,48168317.0,107255683.0,279120,5.787272e+07,7.714168e+07,6.888995e+07,1.337576,1.621011,2.603993,228.150700,35.560117,0.828724,0.787984,1.242133,1.203834,1.244569,3.077369,4.014341,3.773595,2.285791,0.929716,0.463768,0.447478,0.314451,0.253461,229.40,USD,229.40,USD,229.40,USD,229.40,USD,229.40,USD,228.7800,USD,228.7800,USD,228.7800,USD,228.7800,USD,228.7800,USD,228.780,USD,229.40,USD,229.40,USD,229.40,USD,2.380652e+06,USD,228.66600,228.618137,228.45500,228.363077,228.754016,228.482325,228.210634,228.586687,228.367505,228.148323,US0378331005,0.787984,2024-05-02,2024-08-01,Buy,214.883590,USD,Strong buy
1,MSFT,Microsoft Corporation,Packaged software,Technology services,NASDAQ,"S&P 500, NASDAQ 100, Dow Jones Industrial Average, NASDAQ Composite, S&P 500 Information Technology, S&P 100, Russell 3000, Dow Jones Composite Average, NASDAQ-100 Technology Sector, Russell 1000, NASDAQ Computer, S&P 500 ESG, NYSE Arca Major Market",3.415442e+12,USD,459.54,USD,461.00,USD,1.5800,USD,0.343822,0.317709,7.322691e+09,17228192.0,30190828.0,48451,1.641891e+07,1.832156e+07,1.937153e+07,2.037118,1.332034,1.436073,461.623333,39.817350,1.362789,1.057758,1.035710,1.059656,0.893789,2.593111,4.930562,4.519243,2.955850,1.575989,0.952972,0.903491,0.350193,0.189136,468.35,USD,468.35,USD,468.35,USD,468.35,USD,468.35,USD,459.5879,USD,459.5879,USD,459.5879,USD,459.5879,USD,459.5879,USD,460.450,USD,467.33,USD,467.70,USD,468.35,USD,9.588914e+05,USD,459.28300,458.961875,459.00024,459.286500,459.564636,458.881345,458.198054,459.900936,4

In [9]:
#### CREATE MARKET CAP CATEGORIES ####
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans - 200 billion and above
        (df['Market capitalization'] >= 200000000000),
        # Large Cap - 10 billion to 200 billion
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),
        # Midlers - 2 billion to 10 billion
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),
        # Small Cap - 300 million to 2 billion
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000), 
        # Micro Cap - 50 million to 300 million
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),
        # Shrimp - 50 million and below
        (df['Market capitalization'] <= 50000000)
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df

In [10]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)
print(len(trading_view_raw_df.index))

1244


In [11]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [12]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 1244 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Small caps,271,21.784566
1,Large caps,268,21.543408
2,Shrimp,242,19.453376
3,Midlers,236,18.971061
4,Micro caps,185,14.871383
5,Titans,30,2.411576
6,Undefined,12,0.964630


In [13]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(5)

372 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Relative Volume at Time,Relative Volume 1 day,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 week,Relative Volume 1 month,Highest high 1 month,Highest high 1 month - Currency,Highest high 3 months,Highest high 3 months - Currency,Highest high 6 months,Highest high 6 months - Currency,Highest high 52 weeks,Highest high 52 weeks - Currency,Highest high All Time,Highest high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes,marketCapType
4,GOOGL,Alphabet Inc.,Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500 Communication Services, S&P 100, Russell 3000, NASDAQ-100 Technology Sector, Russell 1000, NASDAQ Computer, S&P 500 ESG",2.343616e+12,USD,188.98,USD,189.80,USD,0.3300,USD,0.174622,0.433908,5.859062e+09,15141295.0,36177198.0,31053,19184518.0,2.276644e+07,2.754444e+07,1.398898,1.575929,1.725243,189.686667,28.983329,0.836509,0.776469,1.362278,1.069403,1.047270,3.435862,6.339327,5.050244,2.389596,0.995341,0.469077,0.458346,0.315334,0.171393,191.36,USD,191.36,USD,191.36,USD,191.36,USD,191.36000,USD,189.07,USD,189.0800,USD,189.080,USD,189.080,USD,189.080,USD,189.775,USD,191.36,USD,191.36,USD,191.36,USD,1.683034e+06,USD,188.9746,188.949750,188.941,189.014108,189.030376,188.915155,188.799934,189.664664,189.16226,188.659856,US02079K3059,0.776469,2024-04-25,2024-07-23,Buy,194.062115,USD,Sell,Titans
431,IQ,"iQIYI, Inc.",Internet software/Services,Technology services,NASDAQ,"NASDAQ Composite, NASDAQ Golden Dragon China, NASDAQ Industrials",3.674686e+09,USD,3.83,USD,3.82,USD,0.0008,USD,0.020888,-0.261097,5.246870e+08,6235225.0,14153844.0,31838,7605229.9,8.643044e+06,8.925858e+06,5.785124,5.382219,4.077353,3.766667,13.663932,0.790561,0.845707,1.163895,0.239994,0.071290,3.958597,8.951774,7.757384,3.898555,1.463514,0.670347,0.619015,0.295687,0.254242,4.32,USD,5.80,USD,5.80,USD,6.47,USD,46.23000,USD,3.84,USD,3.8400,USD,3.840,USD,3.840,USD,3.840,USD,3.840,USD,3.84,USD,3.84,USD,3.93,USD,9.395555e+05,USD,3.8170,3.812500,3.816,3.812308,3.830817,3.812250,3.793683,3.830115,3.80975,3.789385,US46267X1081,0.845707,2024-05-16,2024-08-20,Buy,6.114935,USD,Strong buy,Midlers
581,UPWK,Upwork Inc.,Personnel services,Commercial services,NASDAQ,"NASDAQ Composite, Russell 2000, Russell 3000, NASDAQ Computer, Mini-Russell 2000",1.508793e+09,USD,11.38,USD,11.49,USD,0.0700,USD,0

In [14]:
print(len(trading_view_raw_df))

1244


In [15]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

print(tv_df.shape)

(372, 91)


In [16]:
market_info_df = tv_df['marketCapType'].value_counts().reset_index()
market_info_df.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_info_df['percentage'] = (market_info_df['count'] / market_info_df['count'].sum()) * 100

print(market_info_df.shape)
market_info_df

(6, 3)


,marketCapType,count,percentage
0,Large caps,158,42.473118
1,Midlers,107,28.763441
2,Small caps,75,20.161290
3,Titans,26,6.989247
4,Micro caps,5,1.344086
5,Shrimp,1,0.268817


In [17]:
# need to add error handling for when zero (0) return (4/4/24)
# zero again 5/24/24
# - not so much error handling but want to see if we need to adjust the criteria in general
# - need to put this theory and algo through more paper trading and adjust as needed
# think about adding other staples to dataframe
# or you create a separate investments_long_df or similar for those, i.e., TSLA, PLTR, SOFI etc...

| FILTER CRITERIA CONFIG |
|-|
| TradingView (raw data) | 

things to add;
* Pre-Market Volume: Adding a minimum pre-market volume threshold can help filter out stocks that might have large percentage changes but low trading volumes, which can lead to unreliable price movements and high slippage.
Example: A minimum of 50,000 shares traded in the pre-market for all categories.

* overall market sentiment and filter tightener (macro)

In [18]:
#### CRITERIA CONFIG DICTIONARY ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # 1 billion shares
        "relative_volume_threshold": 1.2,  # More inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Mid caps": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        #last
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

In [19]:
# what about exceptions, like if all these criteria above are met but there is super high volume and less than x& < 1% gap-up
# would have to specify specific criteria for the market caps

In [20]:
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    df.loc[:, 'Float shares outstanding'] = pd.to_numeric(df['Float shares outstanding'], errors='coerce')
    return df[df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df.loc[:, 'Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    df.loc[:, 'Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vwap_drawdown_threshold):
    """Filter stocks where the price is near the VWAP, considering the specified drawdown threshold."""
    df.loc[:, 'Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df.loc[:, 'Volume Weighted Average Price 1 day'] = pd.to_numeric(df['Volume Weighted Average Price 1 day'], errors='coerce')
    df.loc[:, 'Min Price from VWAP'] = df['Volume Weighted Average Price 1 day'] * (1 - pre_market_vwap_drawdown_threshold)
    return df[df['Price'] >= df['Min Price from VWAP']].drop(columns=['Min Price from VWAP'])

def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    df.loc[:, 'Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df.loc[:, 'Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df.loc[:, 'Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    return df[(df['Volatility 1 day'] >= df['Volatility 1 week']) & (df['Volatility 1 day'] >= df['Volatility 1 month'])]

#### last (i.e., stay organized, bookend filter section here, add other filters above ####
def filter_by_pre_market_volume(df, pre_market_volume_threshold):
    """Filter stocks by pre-market volume."""
    df['Pre-market Volume'] = pd.to_numeric(df['Pre-market Volume'], errors='coerce')
    return df[df['Pre-market Volume'] >= pre_market_volume_threshold]

In [21]:
# WRAPPER FUNCTION
def screen_stocks_by_category(df, category):
    # FILTERS
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vwap_drawdown_threshold = config.get("pre_market_vwap_drawdown_threshold", 0)
    # last
    pre_market_volume_threshold = config.get("pre_market_volume_threshold", 0)

    # FUNCTIONS (core processing)
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df)
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vwap_drawdown_threshold)
    # last
    filtered_df = filter_by_pre_market_volume(filtered_df, pre_market_volume_threshold)

    return filtered_df

In [22]:
#### EXECUTE AND CREATE FULL (baseline) GAP-UP DATAFRAME ####
gap_up_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    gap_up_df = pd.concat([gap_up_df, gap_up_stage_df])

# filter in descending order by market cap and then by pre-market change percentage
gap_up_df = gap_up_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
gap_up_df.reset_index(drop=True, inplace=True)

# # display results
# print(f'Original dataFrame length: {len(tv_df.index)}')
# print(f'Number of rows removed: {len(tv_df.index) - len(gap_up_df.index)}')
# filter in descending order by market cap and then by pre-market change percentage
gap_up_df = gap_up_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# print(f'NEW (Gap-Up) dataFrame length: {len(gap_up_df.index)} (target stocks)')

# reset index and drop
gap_up_df.reset_index(drop=True, inplace=True)

## LOOK AT COLUMNS WHEN NEEDED
# print(gap_up_df.columns)
## all dataframe view
# gap_up_df

| SCREEN LEVEL I |
|-|
| First TARGETs | 

In [23]:
#### Gap-Up (potential, shortlisted) TARGETs ####
print(f'{len(gap_up_df.index)} Stocks')
gap_up_df.iloc[:, [0, 1, 8]].head(20)

18 Stocks


,Symbol,Description,Price
0,MMYT,MakeMyTrip Limited,90.72
1,WIX,Wix.com Ltd.,156.79
2,YMM,Full Truck Alliance Co. Ltd.,8.21
3,FHN,First Horizon Corporation,15.51
4,AGNC,AGNC Investment Corp.,9.74
5,WAL,Western Alliance Bancorporation,62.43
6,ETSY,"Etsy, Inc.",57.01
7,ZION,Zions Bancorporation N.A.,43.91
8,STWD,Starwood Property Trust Inc.,19.08
9,ALGM,"Allegro MicroSystems, Inc.",31.09


In [149]:
#### CREATE SIMPLE GAP-UP DATAFRAME ####
simple_cols_list = ['Symbol', 'Description', 'Pre-market Change %', 'Pre-market Gap %', 'marketCapType', 'Price', 'Pre-market Open', 'Industry', 'Index', 'Sector', 'Exchange']
simple_gap_df = gap_up_df[simple_cols_list]

# print(simple_gap_df.columns)
first_list = simple_gap_df['Symbol'].unique().tolist()
print(f'{len(simple_gap_df)} (target stocks) TODAY') # fix this to dynamic
print(first_list)
simple_gap_df 

18 (target stocks) TODAY
['MMYT', 'WIX', 'YMM', 'FHN', 'AGNC', 'WAL', 'ETSY', 'ZION', 'STWD', 'ALGM', 'MNSO', 'RITM', 'ACLS', 'ZI', 'COLB', 'AEO', 'PTEN', 'INTR']


,Symbol,Description,Pre-market Change %,Pre-market Gap %,marketCapType,Price,Pre-market Open,Industry,Index,Sector,Exchange
0,MMYT,MakeMyTrip Limited,2.821869,1.410935,Midlers,90.72,92.00,Internet software/Services,"NASDAQ Composite, NASDAQ Industrials",Technology services,NASDAQ
1,WIX,Wix.com Ltd.,1.409529,1.186300,Midlers,156.79,158.65,Information technology services,"NASDAQ Composite, NASDAQ Computer",Technology services,NASDAQ
2,YMM,Full Truck Alliance Co. Ltd.,0.365408,0.365408,Midlers,8.21,8.24,Packaged software,NASDAQ Golden Dragon China,Technology services,NYSE
3,FHN,First Horizon Corporation,0.193424,0.000000,Midlers,15.51,15.51,Regional banks,"S&P MidCap 400, KBW NASDAQ Bank, Russell 3000, Russell 1000",Finance,NYSE
4,AGNC,AGNC Investment Corp.,0.308008,0.205339,Midlers,9.74,9.76,Real estate investment trusts,"NASDAQ Composite, Russell 3000, Russell 1000, NASDAQ Real Estate and Other Financial Services",Finance,NASDAQ
5,WAL,Western Alliance Bancorporation,0.032036,0.032036,Midlers,62.43,62.45,Regional banks,"KBW NASDAQ Bank, Russell 3000, Russell 1000",Finance,NYSE
6,ETSY,"Etsy, Inc.",0.052622,0.508683,Midlers,57.01,57.30,Internet retail,"S&P 500, NASDAQ Composite, S&P 500 Consumer Discretionary, Russell 3000, Russell 1000, NASDAQ Industrials, S&P 500 ESG",Retail trade,NASDAQ
7,ZION,Zions Bancorporation N.A.,0.865407,0.478251,Midlers,43.91,44.12,Major banks,"NASDAQ Composite, NASDAQ Bank, S&P MidCap 400, KBW NASDAQ Bank, Russell 3000, Russell 1000",Finance,NASDAQ
8,STWD,Starwood Property Trust Inc.,0.209644,0.366876,Midlers,19.08,19.15,Real estate investment trusts,"S&P MidCap 400, Russell 3000, Russell 1000",Finance,NYSE
9,ALGM,"Allegro MicroSystems, Inc.",0.611129,0.353812,Midlers,31.09,31.20,Semiconductors,"NASDAQ Composite, PHLX Semiconductor Sector, S&P MidCap 400, Russell 3000, Russell 1000, NASDAQ Computer",Electronic technology,NASDAQ


| SCREEN LEVEL II |
|-|
| Secondary TARGETs |

In [ ]:
# visually verifying is fine
# but we have to put some other reliable mechanism here to create more conviction from screen L1 -> screen L2 -> screen L3

In [148]:
#### ---- DAY TARGETS ---- ####
secondary_list = ['ACLS', 'YMM', 'MNSO', 'AEO', 'ZI']

# create day_df from simple gap df applying a filter matching symbols in day_target_list
second_df = simple_gap_df[simple_gap_df['Symbol'].isin(secondary_list)]

second_df


,Symbol,Description,Pre-market Change %,Pre-market Gap %,marketCapType,Price,Pre-market Open,Industry,Index,Sector,Exchange
2,YMM,Full Truck Alliance Co. Ltd.,0.365408,0.365408,Midlers,8.21,8.24,Packaged software,NASDAQ Golden Dragon China,Technology services,NYSE
10,MNSO,MINISO Group Holding Limited,2.481122,2.481122,Midlers,18.54,19.00,Specialty stores,NASDAQ Golden Dragon China,Retail trade,NYSE
12,ACLS,"Axcelis Technologies, Inc.",1.582153,0.284655,Midlers,151.06,151.49,Industrial machinery,"NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",Producer manufacturing,NASDAQ
13,ZI,ZoomInfo Technologies Inc.,0.749800,0.400962,Midlers,12.47,12.52,Packaged software,"NASDAQ Composite, S&P MidCap 400, Russell 3000, Russell 1000, NASDAQ Computer",Technology services,NASDAQ
15,AEO,"American Eagle Outfitters, Inc.",0.450174,0.000000,Midlers,20.17,20.17,Apparel/Footwear retail,"Russell 2000, Russell 3000, Mini-Russell 2000",Retail trade,NYSE


| SCREEN LEVEL III |
|-|
| Primary TARGETs (PTs) |
| PTs >= 2 |

In [ ]:
# do i need a time var here for the pre-market prices?
# other apis for pre-market and nasdaq registration
# iex api deprecating in August 2024

In [150]:
# return primary targets from df
pt_list = ['ACLS', 'MNSO']
pt_df = simple_gap_df[simple_gap_df['Symbol'].isin(pt_list)]

print(pt_list)
print(pt_df.shape)
pt_df

['ACLS', 'MNSO']
(2, 11)


,Symbol,Description,Pre-market Change %,Pre-market Gap %,marketCapType,Price,Pre-market Open,Industry,Index,Sector,Exchange
10,MNSO,MINISO Group Holding Limited,2.481122,2.481122,Midlers,18.54,19.00,Specialty stores,NASDAQ Golden Dragon China,Retail trade,NYSE
12,ACLS,"Axcelis Technologies, Inc.",1.582153,0.284655,Midlers,151.06,151.49,Industrial machinery,"NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",Producer manufacturing,NASDAQ


In [145]:
#### ---- Quickstart Backtest ---- ####
# manually set target list (>= 2 stocks)
manual_day_list = ['ACLS', 'MNSO',]

print(first_list)
print(secondary_list)
# print(manual_day_list)
print(pt_list)

['MMYT', 'WIX', 'YMM', 'FHN', 'AGNC', 'WAL', 'ETSY', 'ZION', 'STWD', 'ALGM', 'MNSO', 'RITM', 'ACLS', 'ZI', 'COLB', 'AEO', 'PTEN', 'INTR']
['ACLS', 'YMM', 'MNSO', 'AEO', 'ZI']
['ACLS', 'MNSO']


In [27]:
# return a new dataframe with only the stocks in the manual_day_list
manual_day_df = simple_gap_df[simple_gap_df['Symbol'].isin(manual_day_list)]
# use iloc to select first two cols
manual_day_df = manual_day_df.iloc[:, [0, 1, 8]]

manual_day_df

,Symbol,Description,Index
10,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China
12,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000"


In [28]:
#### ---- Quick Stary Backtesting ---- ####
def add_stock_data(df):
    symbols = df['Symbol'].tolist()
    stock_data = {}

    for symbol in symbols:
        stock_info = yf.Ticker(symbol)
        today_data = stock_info.history(period='1d')
        
        if not today_data.empty:
            stock_data[symbol] = {
                'High': today_data['High'].values[0],
                'Low': today_data['Low'].values[0],
                'Open': today_data['Open'].values[0],
                'Close': today_data['Close'].values[0],
                'Date': today_data.index[0]
            }
        else:
            stock_data[symbol] = {
                'High': None,
                'Low': None,
                'Open': None,
                'Close': None,
                'Date': None
            }

    # create a DataFrame from the stock data dictionary
    stock_data_df = pd.DataFrame(stock_data).T.reset_index().rename(columns={'index': 'Symbol'})
    
    # merge the new stock data with the existing DataFrame
    updated_df = pd.merge(df, stock_data_df, on='Symbol', how='left')
    
    return updated_df

# fetch prices at specific times
def add_intraday_prices(df, interval='1m'):
    symbols = df['Symbol'].tolist()
    intraday_data = []

    for symbol in symbols:
        stock_info = yf.Ticker(symbol)
        today_intraday = stock_info.history(interval=interval, period='1d')
        
        # convert the index to Eastern time zone (eastern is default for NYSE and NASDAQ)
        today_intraday.index = today_intraday.index.tz_convert('US/Eastern')
        
        # find the closest times to 09:33 PST and 07:15 PST
        price_0933 = today_intraday.between_time('09:33', '09:34')['Close']
        price_1015 = today_intraday.between_time('10:15', '10:16')['Close']
        
        # use the first available price in the range if not empty
        price_0933_value = price_0933.values[0] if not price_0933.empty else None
        price_1015_value = price_1015.values[0] if not price_1015.empty else None
        
        intraday_data.append({
            'Symbol': symbol,
            'Price@09:33': price_0933_value,
            'Price@10:15': price_1015_value
        })

    # create a df from intraday data list
    intraday_data_df = pd.DataFrame(intraday_data)
    
    return intraday_data_df

# add stock data to existing df
qstart_bt_df = add_stock_data(manual_day_df)

# fetch intraday prices and merge with the existing df
intraday_prices_df = add_intraday_prices(manual_day_df)
qstart_bt_df = pd.merge(qstart_bt_df, intraday_prices_df, on='Symbol', how='left')


display(qstart_bt_df)

,Symbol,Description,Index,High,Low,Open,Close,Date,Price@09:33,Price@10:15
0,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China,19.549999,18.93,19.1,19.16,2024-07-10 00:00:00-04:00,19.075001,19.229099
1,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",158.610001,151.669998,153.449997,155.020004,2024-07-10 00:00:00-04:00,153.199997,153.199997


In [29]:
# create staging df for high and low prices
high_low_time_df = qstart_bt_df[['Symbol', 'Description', 'High', 'Low']]

print(high_low_time_df)

  Symbol Description                   High        Low        
0  MNSO   MINISO Group Holding Limited   19.549999       18.93
1  ACLS     Axcelis Technologies, Inc.  158.610001  151.669998


In [30]:
# fetch the timestamps @ high and low price marks
def get_high_low_timestamps(symbol, interval='1m', period='1d'):
    data = yf.Ticker(symbol).history(interval=interval, period=period)
    high_time_stamp = data['High'].idxmax()
    low_time_stamp = data['Low'].idxmin()
    return high_time_stamp, low_time_stamp

# fetch timestamps and update the df
high_low_time_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df['Symbol'].apply(
    lambda symbol: pd.Series(get_high_low_timestamps(symbol))
)

print(high_low_time_df)

  Symbol Description                   High        Low         high_time_stamp           low_time_stamp           
0  MNSO   MINISO Group Holding Limited   19.549999       18.93 2024-07-10 09:48:00-04:00 2024-07-10 15:37:00-04:00
1  ACLS     Axcelis Technologies, Inc.  158.610001  151.669998 2024-07-10 09:45:00-04:00 2024-07-10 09:30:00-04:00


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/118328880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df['Symbol'].apply(
/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/118328880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df['Symbol'].apply(


In [31]:
# transform time string data
high_low_time_df['high_time_stamp'] = high_low_time_df['high_time_stamp'].dt.time
high_low_time_df['low_time_stamp'] = high_low_time_df['low_time_stamp'].dt.time

high_low_time_df

/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/2157241607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df['high_time_stamp'] = high_low_time_df['high_time_stamp'].dt.time
/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_68713/2157241607.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low_time_df['low_time_stamp'] = high_low_time_df['low_time_stamp'].dt.time


,Symbol,Description,High,Low,high_time_stamp,low_time_stamp
0,MNSO,MINISO Group Holding Limited,19.549999,18.93,09:48:00,15:37:00
1,ACLS,"Axcelis Technologies, Inc.",158.610001,151.669998,09:45:00,09:30:00


In [32]:
# add the last two cols to the qstart_bt_df
qstart_bt_df[['high_time_stamp', 'low_time_stamp']] = high_low_time_df[['high_time_stamp', 'low_time_stamp']]

# transform Date column string to set the format
qstart_bt_df['Date'] = pd.to_datetime(qstart_bt_df['Date']).dt.strftime('%Y-%m-%d')

print(qstart_bt_df.shape)
qstart_bt_df

,Symbol,Description,Index,High,Low,Open,Close,Date,Price@09:33,Price@10:15,high_time_stamp,low_time_stamp
0,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China,19.549999,18.93,19.1,19.16,2024-07-10,19.075001,19.229099,09:48:00,15:37:00
1,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",158.610001,151.669998,153.449997,155.020004,2024-07-10,153.199997,153.199997,09:45:00,09:30:00


In [34]:
#### ---- Set Manual Trade Details ---- ####
# structured dictionary for two (2) stocks with buy and sell details
trade_deets = {
    'MNSO': {
        'buy_price': 19.11,
        'buy_time': '09:31:47',
        'quantity': 300,
        'sell_price': 19.40,
        'sell_time': '09:58:19',
        'trading_platform': 'Fidelity'
    },
      'ACLS': {
        'buy_price': 153.20,
        'buy_time': '09:31:10',
        'quantity': 50,
        'sell_price': 158.24,
        'sell_time': '09:55:00',
        'trading_platform': 'Robinhood'
    }
}

In [35]:
# add trade details to the DataFrame
for symbol in trade_deets:
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'buy_price'] = trade_deets[symbol]['buy_price']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'buy_time'] = trade_deets[symbol]['buy_time']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'quantity'] = trade_deets[symbol]['quantity']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'sell_price'] = trade_deets[symbol]['sell_price']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'sell_time'] = trade_deets[symbol]['sell_time']
    qstart_bt_df.loc[qstart_bt_df['Symbol'] == symbol, 'trading_platform'] = trade_deets[symbol]['trading_platform']

print(qstart_bt_df.shape)
qstart_bt_df

(2, 18)


,Symbol,Description,Index,High,Low,Open,Close,Date,Price@09:33,Price@10:15,high_time_stamp,low_time_stamp,buy_price,buy_time,quantity,sell_price,sell_time,trading_platform
0,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China,19.549999,18.93,19.1,19.16,2024-07-10,19.075001,19.229099,09:48:00,15:37:00,19.11,09:31:47,300.0,19.40,09:58:19,Fidelity
1,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",158.610001,151.669998,153.449997,155.020004,2024-07-10,153.199997,153.199997,09:45:00,09:30:00,153.20,09:31:10,50.0,158.24,09:55:00,Robinhood


In [36]:
# add math calculation columns
qstart_bt_df['amount@buy'] = qstart_bt_df['buy_price'] * qstart_bt_df['quantity']
qstart_bt_df['amount@close'] = qstart_bt_df['sell_price'] * qstart_bt_df['quantity']
qstart_bt_df['realized_$'] = qstart_bt_df['amount@close'] - qstart_bt_df['amount@buy']
qstart_bt_df['realized_%'] = (qstart_bt_df['realized_$'] / qstart_bt_df['amount@buy']) * 100

print(qstart_bt_df.shape)
qstart_bt_df

(2, 22)


,Symbol,Description,Index,High,Low,Open,Close,Date,Price@09:33,Price@10:15,high_time_stamp,low_time_stamp,buy_price,buy_time,quantity,sell_price,sell_time,trading_platform,amount@buy,amount@close,realized_$,realized_%
0,MNSO,MINISO Group Holding Limited,NASDAQ Golden Dragon China,19.549999,18.93,19.1,19.16,2024-07-10,19.075001,19.229099,09:48:00,15:37:00,19.11,09:31:47,300.0,19.40,09:58:19,Fidelity,5733.0,5820.0,87.0,1.517530
1,ACLS,"Axcelis Technologies, Inc.","NASDAQ Composite, Russell 2000, PHLX Semiconductor Sector, Russell 3000, NASDAQ Computer, Mini-Russell 2000",158.610001,151.669998,153.449997,155.020004,2024-07-10,153.199997,153.199997,09:45:00,09:30:00,153.20,09:31:10,50.0,158.24,09:55:00,Robinhood,7660.0,7912.0,252.0,3.289817


In [38]:
# save day of week as a string
day_of_week = datetime.datetime.today().strftime('%A')
# lowercase and only first 3 chars
day_of_week_abrv = day_of_week[:3]

print(day_of_week)
print(day_of_week_abrv)

Wednesday
Wed


In [39]:
# create or update existing directory
trade_logz_dir = "tRaDeLoGz"
os.makedirs(trade_logz_dir, exist_ok=True)

# save to xlsx in that directory
trade_logz_dir_file_path = f"{trade_logz_dir}/tRaDeLoGz_{day_of_week_abrv}{today_date_str}.xlsx"
qstart_bt_df.to_excel(trade_logz_dir_file_path, index=False)

In [40]:
# open the file
os.system(f"open {trade_logz_dir_file_path}")

0

In [ ]:
# START here with the trade details
# later figure out how you can speed things up with news
# keep automating and speeding things up
# reorder the columns
# you can keep improving it every day if you want to, but you will have the trade record avaialable data saved for your records
# next iteration you have to rearrange cols, set sheet number
# also set the col var types properly, i.e., for $ and % and probably others
# should also have some kinda notes.
# maybe even ideal trade?? not sure, isn't that obvious it would be the high and low of the day
# accompany these time stamps with volume somehow
# thinking later of using volume to determine the best time to sell instead of times (which is based on general intraday (early 630-730) volume trends)
# can get fancier with excel output maybe set the zoom, auto open, not sure, could also get as fancy as you want with excel wings
# keep the one file but different excel docouments for each day, for now just one sheet per doc keep simple
# could include type of buy and sell
# should figure out strategy, i.e., some standard type protocols like trailing stop lostt % not sure
# would like to do percenct, maybe use market cap or price to determine bc prob cant be standard for all stocks all time, maybe tho

#### the 2 you really trade. all others you should paper trade and measure, then keep refining the program as needed
# seek advice and outside input

In [41]:
#### ---- Create Folders ---- ####
# also create excel doc for each trading day, checks for existing folders and files and update or add to them
# reorder cols later
# you did this over in the edgartools, use that to start and copy

In [42]:
# # save day_df to csv
# day_df.to_csv(f"trading_view_raw_data/tv_screen_gap-up_2024-07-10_612_day_df.csv", index=False)


In [43]:
#### ---- TRADING NOTES -> DATE = 7/10/24 ---- ####
""" 
MINISO Group Holding Limited (MNSO) - High likelihood to gap up due to strong pre-market activity, positive news, and robust financial performance.
Axcelis Technologies, Inc. (ACLS) - High likelihood to gap up driven by strong industry performance, positive financial results, and favorable technical indicators.
"""

' \nMINISO Group Holding Limited (MNSO) - High likelihood to gap up due to strong pre-market activity, positive news, and robust financial performance.\nAxcelis Technologies, Inc. (ACLS) - High likelihood to gap up driven by strong industry performance, positive financial results, and favorable technical indicators.\n'

In [44]:
# here is where you are at. 
# aLSO I WANT A is it a good pre-market score before the main screener and then after also accounting for main screener and all pre screeners

In [45]:
#### you could try in the morning to get IEX news
# maybe a try with the foreign news, IF foreign exchanges = true?
# could also look at the indexes returned in the screener and then chart those after also
# might need to create symbol and name dictionary to work with

# need price
# news
# other markets
# news and translation
# index comparison
# trends

## backtesting 
# openprice, price at 5min, price at 0705, (perc gain), high, low, close
# from here can also indicate shares simulated, etc...

| Backtesting -> START |
|-|
| HEAVY Current DEV |
| < July 4th |

In [46]:
print(sgap_stock_list)

['MMYT', 'WIX', 'YMM', 'FHN', 'AGNC', 'WAL', 'ETSY', 'ZION', 'STWD', 'ALGM', 'MNSO', 'RITM', 'ACLS', 'ZI', 'COLB', 'AEO', 'PTEN', 'INTR']


In [47]:
# # List of stock symbols
# symbols = ['SFM', 'ATI', 'AAON', 'MNSO', 'LNTH', 'CVLT', 'CRS', 'SMPL', 'TRMD', 'SDRL', 'UNF', 'GOGL', 'VZIO', 'PAYO']

# # Specify the date and times (in EST)
# date_str = '2024-06-26'  # example date
# # date_str = datetime.now().strftime('%Y-%m-%d')
# times_est = ['09:35', '10:05']  # times in EST

# # Fetch historical data and extract prices at specified times
# data = {'Symbol': [], '$@0935': [], '$@1005': []}

# for symbol in symbols:
#     ticker = yf.Ticker(symbol)
#     # Fetch 1-minute data for the specified date
#     hist = ticker.history(start=date_str, end=(datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1m')
    
#     # Ensure the index is timezone-aware by localizing to UTC if necessary, then convert to EST
#     if hist.index.tz is None:
#         hist.index = hist.index.tz_localize('UTC').tz_convert('US/Eastern')
#     else:
#         hist.index = hist.index.tz_convert('US/Eastern')

#     # Filter data for specified times
#     price_at_0935 = hist.loc[hist.index.strftime('%H:%M') == '09:35']['Close'].values
#     price_at_1035 = hist.loc[hist.index.strftime('%H:%M') == '10:35']['Close'].values
    
#     # Save data
#     data['Symbol'].append(symbol)
#     data['$@0935'].append(price_at_0935[0] if len(price_at_0935) > 0 else None)
#     data['$@1005'].append(price_at_1035[0] if len(price_at_1035) > 0 else None)

# # Convert to DataFrame for better visualization
# df = pd.DataFrame(data)

# print(date_str)
# df

In [48]:
# symbols = ['SFM', 'ATI', 'AAON', 'MNSO', 'LNTH', 'CVLT', 'CRS', 'SMPL', 'TRMD', 'SDRL', 'UNF', 'GOGL', 'VZIO', 'PAYO']

# # Specify the date (in EST)
# # date_str = '2024-06-26'  # example date
# date_str = datetime.now().strftime('%Y-%m-%d')

# # Fetch historical data and extract Open, Close, High, and Low prices
# data_daily = {'Symbol': [], 'Open': [], 'Close': [], 'High': [], 'Low': []}

# for symbol in symbols:
#     ticker = yf.Ticker(symbol)
    
#     # Fetch daily data for the specified date to get Open, Close, High, and Low prices
#     hist_daily = ticker.history(start=date_str, end=(datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1d')
    
#     # Get the daily Open, Close, High, and Low prices
#     open_price = hist_daily['Open'].values
#     close_price = hist_daily['Close'].values
#     high_price = hist_daily['High'].values
#     low_price = hist_daily['Low'].values
    
#     # Save daily data
#     data_daily['Symbol'].append(symbol)
#     data_daily['Open'].append(open_price[0] if len(open_price) > 0 else None)
#     data_daily['Close'].append(close_price[0] if len(close_price) > 0 else None)
#     data_daily['High'].append(high_price[0] if len(high_price) > 0 else None)
#     data_daily['Low'].append(low_price[0] if len(low_price) > 0 else None)

# # Convert to DataFrame for better visualization
# df_daily = pd.DataFrame(data_daily)

# print(date_str)
# df_daily

| Backtesting <- END |
|-|
| HEAVY Current DEV |
| < July 4th |

In [49]:
# # to csv
# simple_gap_df.to_csv(f"trading_view_raw_data/UPD_{today_date_str}", index=False)

| vectorb Development |
|-|
| START DEV |

In [50]:
# #### Yahoo Finance API ####
# # yf_vbt_stock = 'NOW' # servicenow
# yf_vbt_stock = sgap_stock_list[0] # first stock in the list
# print(f'${yf_vbt_stock}')
# yahoo_price_data = yf.Ticker(yf_vbt_stock).history(period='1d')

# yahoo_price_close = yahoo_price_data['Close'].iloc[-1]
# yahoo_price_date = yahoo_price_data.index[-1]

# print(f'${yahoo_price_close}')
# print(yahoo_price_date)


In [51]:
# iex_api_key = os.getenv("IEX_API_KEY")  # Replace with your actual IEX API key

In [52]:
# # set iex api parameters
# iex_symbol = 'NOW'
# iex_mins_interval = '10'
# # iex_day_range = '10d'

# # iex url construct
# url = f'https://api.iex.cloud/v1/data/core/minutebar/{iex_symbol}?last={iex_mins_interval}&token={iex_api_key}'

# # fetch data
# response = requests.get(url)
# iex_data = response.json()
# print(response.status_code)

# # look at the json
# print(iex_data)
# # # check if data is in expected format and convert to DataFrame
# # if isinstance(iex_data, list):
# #     iex_hd_df = pd.DataFrame(iex_data)
# #     print(iex_hd_df)

In [53]:
# if isinstance(iex_data, list):
#     parsed_data = []

#     # Parse the JSON data
#     for item in iex_data:
#         for time, data in item.items():
#             if time not in ['id', 'key', 'subkey', 'date', 'updated']:
#                 data['time'] = time
#                 parsed_data.append(data)

#     # Convert to DataFrame
#     iex_hd_df = pd.DataFrame(parsed_data)
#     iex_hd_df['date'] = pd.to_datetime(iex_hd_df['date'], format='%Y%m%d')
#     iex_hd_df['datetime'] = iex_hd_df['date'].astype(str) + ' ' + iex_hd_df['time']
#     iex_hd_df['datetime'] = pd.to_datetime(iex_hd_df['datetime'])
#     iex_hd_df.set_index('datetime', inplace=True)
    
#     # Drop the separate date and time columns
#     iex_hd_df.drop(columns=['date', 'time'], inplace=True)

#     # Display the DataFrame
#     print("DataFrame:")
#     print(iex_hd_df)
# else:
#     print("Unexpected data format:", iex_data)

# # view
# iex_hd_df

In [54]:
#### historical data
# determine time FRAMEs 
# determine time RANGE
# retrieve historical data from iex cloud api

# what about market volatility trends, times, lunch power, both daily and cyclically, quarterly, event based like earnings, sector etc.

In [55]:
# # higher frequency
# start_date = '2024-05-24'
# end_date = '2024-05-25'
# vbt_data = vbt.YFData.download('NOW', start=start_date, end=end_date, interval='5m')

In [56]:
# close_5m = vbt_data.get('Close')

# print(close_5m)

In [57]:
# # print the day today - 60 days
# dt.date.today() - dt.timedelta(days=60)

In [58]:
# # symbol = 'NOW'
# symbol = sgap_stock_list[0]
# # set the start date to today -59 days
# start_date = (dt.date.today() - dt.timedelta(days=59)).strftime('%Y-%m-%d')  # Adjust to current date
# end_date = dt.datetime.today().strftime('%Y-%m-%d')  # Adjust to current date

# # download data for each timeframe
# timeframes = ['5m', '15m', '30m', '1h']
# data = {tf: vbt.YFData.download(symbol, start=start_date, end=end_date, interval=tf) for tf in timeframes}

# # calculate SMAs for each timeframe
# fast_sma = {tf: data[tf].get('Close').rolling(window=20).mean() for tf in timeframes}
# slow_sma = {tf: data[tf].get('Close').rolling(window=50).mean() for tf in timeframes}

In [59]:
# # convert to dataframs
# fast_sma_df = pd.DataFrame(fast_sma)
# slow_sma_df = pd.DataFrame(slow_sma)

In [60]:
# fast_sma_df.tail(10)

In [61]:
# print(len(fast_sma_df))
# fast_sma_df.dropna(how='all', inplace=True)
# slow_sma_df.dropna(how='all', inplace=True)
# print(len(fast_sma_df))

| 1-min chart |
|-|

In [62]:
# first_stock = simple_gap_df['Symbol'].iloc[0]
# print(first_stock)


In [63]:
# # symbol = 'NOW'
# symbol = simple_gap_df['Symbol'].iloc[0]
# # auto - dev in progress
# start_date = (dt.date.today() - dt.timedelta(days=7)).strftime('%Y-%m-%d')  
# end_date = dt.datetime.today().strftime('%Y-%m-%d')  

# # download data for each timeframe
# timeframes = ['1m']
# one_min_data = {tf: vbt.YFData.download(symbol, start=start_date, end=end_date, interval=tf) for tf in timeframes}

# # calculate SMAs for each timeframe
# one_min_fast_sma = {tf: one_min_data[tf].get('Close').rolling(window=20).mean() for tf in timeframes}
# one_min_slow_sma = {tf: one_min_data[tf].get('Close').rolling(window=50).mean() for tf in timeframes}

In [64]:
# # convert to dataframs
# one_min_fast_sma_df = pd.DataFrame(one_min_fast_sma)
# one_min_slow_sma_df = pd.DataFrame(one_min_slow_sma)

In [65]:
# # NaNs check and drop
# print(len(one_min_fast_sma_df))
# one_min_fast_sma_df.dropna(inplace=True)
# one_min_slow_sma_df.dropna(inplace=True)
# print(len(one_min_fast_sma_df))

In [66]:
# one_min_fast_sma_df.head(4)

In [67]:
#### merge the dataframes
### add the ticker and description to every row
## could try to get more batches of 7 days
# could also try exploring different apis more

In [68]:
# # vbt_data = vbt.YFData.download(yf_vbt_stock, start='2024-05-01', end='2024-05-31')
# price = vbt.YFData.download('NOW', start='2024-05-01', end='2024-05-31').get('Close')
# # price = vbt.YFData.download(yf_vbt_stock, start='2024-05-01', end='2024-05-31').get('Close')

# # print(vbt_data)
# print(price)

In [69]:
#### action on other exchanges

In [70]:
#### news, foreign exchanges, foreign news, translate.

In [71]:
#### crew ai setup i feed them the best info possible

# personas
# dev = code review and feedback (ea day), potential model tweaks, could be good ci/cd pipeline servicenow demo
# sector analyst
# market analyst
# resarcher 1
# researcher 2
# human

| vectorb Development| 
|-|
| END DEV|

| guss_pl_df |
|-|
| Gap-up strategy screener, profit & loss dataframe |
| START -> guss |

In [72]:
#### simulation plan = buy @ 635, sell @ 705 

In [73]:
# save / update results to excel

| guss_pl_df |
|-|
| END <- guss |

| Indexes |
|-|
| IEX Cloud API $ | 

market indexes, sector indexes
currencies???

In [74]:
# #### STANDARD MARKET INDEXES EVALUATION
# market_index_list = ['SPY', # SPX = S&P 500 index
#                      'RSP', # SPXEW - S&P 500 equal weight index
#                      'XLRE'] # real estate select sector SPDR fund

# #### IEX CLOUD API KEY -> VARIABLE ####
# iex_api_key = os.getenv("IEX_API_KEY")  # Replace with your actual IEX API key

# # Set the list of symbols variable
# index_symbols = ','.join(market_index_list)

# #### SET DATE RANGE HERE for - historical data processing
# data_range = '5m' 

# url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={index_symbols}&types=chart&range={data_range}&token={iex_api_key}'

# # Make the GET request
# response = requests.get(url)

# # Initialize a DataFrame to collect data for each symbol
# market_index_df = pd.DataFrame()

# # Check if the request was successful
# if response.status_code == 200:
#     # Convert the response to JSON
#     data = response.json()

#     # Process and print the data with a progress bar
#     for symbol in tqdm(market_index_list, desc="Processing symbols"):
#         if symbol in data:
#             # Create a DataFrame for the current symbol's data
#             symbol_df = pd.DataFrame(data[symbol]['chart'])
#             # Add a 'symbol' column to identify the data
#             symbol_df['symbol'] = symbol
#             # Append the current symbol's DataFrame to the main DataFrame
#             market_index_df = pd.concat([market_index_df, symbol_df], ignore_index=True)
# else:
#     print(f'Failed to fetch data: {response.status_code}')

# # Ensure date column is datetime format
# market_index_df['date'] = pd.to_datetime(market_index_df['date'])

# #### FILTER matching MANDATORY columns only
# hist_data_mandatory_cols_list = ['symbol', 'date', 'close', 'high', 'low', 'open', 'volume']
# market_index_df = market_index_df[hist_data_mandatory_cols_list]

# # Sort by date descending and ticker order specified in the list
# market_index_df['symbol'] = pd.Categorical(market_index_df['symbol'], categories=market_index_list, ordered=True)
# market_index_df = market_index_df.sort_values(by=['date', 'symbol'], ascending=[False, True])
# # Reset the index to get a numbered index
# market_index_df = market_index_df.reset_index(drop=True)
# # rename the new index (these are DAYS aka trading periods since today if sorted descending)
# # market_index_df.index.name = 'INDEX'

# # print(market_index_df.columns)

# # # view df
# # market_index_df

In [75]:
# # Set variables for historical data analysis
# mi_length = len(market_index_df.index)
# unique_stocks = market_index_df['symbol'].nunique()
# start_date = market_index_df['date'].min()
# end_date = market_index_df['date'].max()

# print(f"START date: {start_date}")
# print(f"END date: {end_date}")
# print(f'{market_index_df["date"].nunique()} days of historical stock data.\n')
# print(f"{mi_length} = Length of dataframe")
# print(f"{unique_stocks} = Unique stocks in dataframe")

# # Entries per stock
# days_per_stock = mi_length / unique_stocks
# print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# # View the stocks
# print(f'Unique stock ticker symbols: {market_index_list}')
# index_cols_list = market_index_df.columns.tolist()
# print(f'Columns: {index_cols_list}')

# # convert df to .csv
# # historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# print('INDEX DATAFRAME: historical price data $')
# # Display the first few rows of the dataframe
# market_index_df.tail(10)

| MARKET SENTIMENT | 
|-|

In [76]:
#### MARKET SENTIMENT
# 1 - VIX (Volatility Index): A high VIX can indicate market uncertainty, making it essential to manage risk carefully.
# 2 - Overall Market Trend: Align your trades with the overall market trend. If major indices are bullish, it supports a positive trading environment.
# 2i - see CRAMER
# 2ii - see CRAMER
# 2iii - see CRAMER
# 3 - maybe some CRAMER + your own adaptation(s)
# 4 - Index match and performance
# 5 - sector index match on performance
# 6 - how do i deal with implied volatility, do i overlay, is there  pre-built metric (if so how is that calculated)
# 7 - further VIX and other adjustments, how to deal with implied volatility, at the macro and specific stock level / sector level and whatever is in between

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [77]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [78]:
"""THIS IS WHERE YOU HAVE TO START AND DO WORK, maybe move up to organize better"""

'THIS IS WHERE YOU HAVE TO START AND DO WORK, maybe move up to organize better'

In [79]:
# """NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
# info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

# info_news_df

In [80]:
# #### IEX CLOUD API KEY -> VARIABLE ####
# iex_api_key = os.getenv("IEX_API_KEY")

# # set the list of symbols variable
# symbols = ','.join(sgap_stock_list)

# # SETTING THE DATE RANGE
# """
# y - Example: range=1y -> returns (one year)
# ytd - Example: range=ytd -> returns (year-to-date)
# m - Example: range=6m returns (six months)
# d - Example: range=5d returns (five days)
# """
# data_range = '5m' # DATE RANGE (for historical stock price data)

# url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_api_key}'

# # Make the GET request
# response = requests.get(url)

# # Initialize a list to collect DataFrames for each symbol
# dataframes_list = []

# # Check if the request was successful
# if response.status_code == 200:
#     # Convert the response to JSON
#     data = response.json()

#     # Process and print the data with a progress bar
#     for symbol in tqdm(sgap_stock_list, desc="Processing symbols"):
#         if symbol in data:
#             # Create a DataFrame for the current symbol's data
#             symbol_df = pd.DataFrame(data[symbol]['chart'])
#             # Add a 'Ticker' column to identify the data
#             symbol_df['Ticker'] = symbol
#             # Append the current symbol's DataFrame to the list
#             dataframes_list.append(symbol_df)
#             # print(f'Data for {symbol}:')
#             # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
# else:
#     print(f'Failed to fetch data: {response.status_code}')

# # concatenate dataframes and create a combined list
# historical_df = pd.concat(dataframes_list, ignore_index=True)


In [81]:
# # Set variables for historical data analysis
# historical_length = len(historical_df.index)
# unique_stocks = historical_df['Ticker'].nunique()
# start_date = historical_df['date'].min()
# end_date = historical_df['date'].max()

# print(f"START date: {start_date}")
# print(f"END date: {end_date}")
# print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
# print(f"{historical_length} = Length of dataframe")
# print(f"{unique_stocks} = Unique stocks in dataframe")

# # Entries per stock
# days_per_stock = historical_length / unique_stocks
# print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# # View the stocks
# print(f'Unique stock ticker symbols: {sgap_stock_list}')
# historical_cols_list = historical_df.columns.tolist()
# print(f'Columns: {historical_cols_list}')

# # convert df to .csv
# # historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# # Display the first few rows of the dataframe
# historical_df.tail(10)

In [82]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
# historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [83]:
# # Ensure the dataframe is sorted by date if it's not already
# historical_sorted_df = historical_df.sort_values(by='date')

# # Select mandatory columns for a historical price bar chart + volume
# # and explicitly copy the dataframe to ensure it's not a view but a new object
# chart_historical_df = historical_sorted_df[[
#     'symbol',   # *MANDATORY
#     'date',     # *MANDATORY
#     'close',    # *MANDATORY
#     'high',     # *MANDATORY
#     'low',      # *MANDATORY
#     'open',     # *MANDATORY
#     'volume'    # *MANDATORY
# ]].copy()

# # Group by 'symbol' and get the last occurrence
# last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

# last_date_df


In [84]:
# # re-index the dataframe
# # chart_historical_df.reset_index(drop=True, inplace=True)
# # view head of your main df
# print(f'Length of historical data: {len(chart_historical_df.index)}')

In [85]:
# you need to get the chart dataframe to a place where it has all the data you need in there. 
# it would be more next level if we let it rip into live stock feeds from here. all setup with the correct lines.
# in the charts live feed we could use the tradingview html widget to display the charts.

In [86]:
"""END DEV FRIDAY"""

'END DEV FRIDAY'

| KEY LEVELS (Support & Resistance) |
|-|
| TECHNICAL INDICATOR CALCULATIONS FROM HISTORICAL DATA |

in order of priority of in scope. 
thinking of moving some of the levels out as i test them and see what they look like.
for instance i am thinking of eventually completely replacing MAs with EMAs

In [87]:
# "calculate key levels from historical data"
# # save historical data to a csv
# # chart_historical_df.to_csv(f'chart_historical_gpt_data{today_date_str}.csv', index=False)

# chart_historical_df.head()
# # think about returning unique values first row 
# # you wont get today if b/c of no close price obviously

| HISTORICAL DATA |
|-|
| Baselne Data: OHCLV (Open, High, Low, Close, Volume) Candlestick Chart |
| IMPORTANT: For effective plotting and time series analysis, it's crucial that the date column in the df is in the datetime format rather than an object (string). |

In [88]:
# chart_historical_df['date'] = pd.to_datetime(chart_historical_df['date'])
# chart_historical_df.set_index('date', inplace=True)

# print(chart_historical_df.index.dtype)  # Should print 'datetime64[ns]'
# print(chart_historical_df.columns)  # This will list all columns except 'date'
# chart_historical_df.head()


# TREND LINES - UNIT TESTING

| TICKER - Unit TESTING | 
|-|

In [89]:
# #### dynamic symbol test update 
# # return the unique symbols and store as list
# unique_symbols = chart_historical_df['symbol'].unique()
# unique_symbols_list = unique_symbols.tolist()

# print(unique_symbols_list)

# ex_ticker = unique_symbols_list[0]

# print(ex_ticker)

In [90]:
# # #### HARDCODED SYMBOL -> Option #1
# # ticker = 'CRM'
# # ticker_df = chart_historical_df[chart_historical_df['symbol'] == ticker].copy()

# #### DYNAMIC SYMBOL (allocation) -> Option #2
# ticker = ex_ticker
# ticker_df = chart_historical_df[chart_historical_df['symbol'] == ex_ticker].copy()

# ticker_df.head()

In [91]:
# # rename columns so they are capitalized for mplfinance specs
# ticker_df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

# ticker_df.tail(10)

In [92]:
# # save unit test dataframe to csv
# ticker_df.to_csv(f'unit_TEST/historical_data_{ticker}{today_date_str}.csv', index=True)

# ADVANCED WIP

In [93]:
# ticker_df.head(6)

| ADVANCED CHARTING | 
|-|
| Trend Lines | 

In [94]:
# def check_trend_line(support: bool, pivot: int, slope: float, y: np.array):
#     # compute sum of differences between line and prices, 
#     # return negative value if invalid 
    
#     intercept = -slope * pivot + y.iloc[pivot]  # Using .iloc[] for positional indexing
#     line_vals = slope * np.arange(len(y)) + intercept  # Calculate line values
#     diffs = line_vals - y  # Calculate differences between line values and actual values
    
#     if support and diffs.max() > 1e-5:
#         return -1.0  # Line is not valid for support
#     elif not support and diffs.min() < -1e-5:
#         return -1.0  # Line is not valid for resistance
    
#     err = (diffs ** 2.0).sum()  # Calculate sum of squared errors
#     return err

# def optimize_slope(support: bool, pivot: int, init_slope: float, y: np.array):
#     slope_unit = (y.max() - y.min()) / len(y)  # Determine the unit change for slope
    
#     # Initialize optimization variables
#     opt_step = 1.0
#     min_step = 0.0001
#     curr_step = opt_step
#     best_slope = init_slope
#     best_err = check_trend_line(support, pivot, init_slope, y)
    
#     assert(best_err >= 0.0)  # Initial slope should not produce an error
    
#     get_derivative = True
#     derivative = None
#     while curr_step > min_step:
#         if get_derivative:
#             # Numerical differentiation to find the direction to adjust slope
#             slope_change = best_slope + slope_unit * min_step
#             test_err = check_trend_line(support, pivot, slope_change, y)
#             derivative = test_err - best_err

#             if test_err < 0.0:  # If increasing slope fails, try decreasing
#                 slope_change = best_slope - slope_unit * min_step
#                 test_err = check_trend_line(support, pivot, slope_change, y)
#                 derivative = best_err - test_err

#             if test_err < 0.0:  # If derivative failed, abort optimization
#                 raise Exception("Derivative failed. Check your data.")

#             get_derivative = False

#         if derivative > 0.0:  # If increasing slope increased error
#             test_slope = best_slope - slope_unit * curr_step
#         else:  # If increasing slope decreased error
#             test_slope = best_slope + slope_unit * curr_step
        
#         test_err = check_trend_line(support, pivot, test_slope, y)
#         if test_err < 0 or test_err >= best_err:  # If slope failed or didn't reduce error
#             curr_step *= 0.5  # Reduce step size
#         else:  # If test slope reduced error
#             best_err = test_err
#             best_slope = test_slope
#             get_derivative = True  # Recompute derivative
    
#     return (best_slope, -best_slope * pivot + y.iloc[pivot])  # Using .iloc[] for positional indexing

# def fit_trendlines_single(data: np.array):
#     x = np.arange(len(data))
#     coefs = np.polyfit(x, data, 1)  # Fit line of best fit
#     line_points = coefs[0] * x + coefs[1]  # Get points of line
    
#     # Find pivot points
#     upper_pivot = (data - line_points).argmax()
#     lower_pivot = (data - line_points).argmin()
   
#     # Optimize the slope for both trend lines
#     support_coefs = optimize_slope(True, lower_pivot, coefs[0], data)
#     resist_coefs = optimize_slope(False, upper_pivot, coefs[0], data)

#     return (support_coefs, resist_coefs)

# def fit_trendlines_high_low(high: np.array, low: np.array, close: np.array):
#     x = np.arange(len(close))
#     coefs = np.polyfit(x, close, 1)  # Fit line of best fit
#     line_points = coefs[0] * x + coefs[1]  # Get points of line

#     # Find pivot points
#     upper_pivot = (high - line_points).argmax()
#     lower_pivot = (low - line_points).argmin()
    
#     # Optimize the slope for both trend lines
#     support_coefs = optimize_slope(True, lower_pivot, coefs[0], low)
#     resist_coefs = optimize_slope(False, upper_pivot, coefs[0], high)

#     return (support_coefs, resist_coefs)


In [95]:
# should be based on 72 periods but displayed as 24 periods maybe? nah?

In [96]:
# # step 1: fit trendlines for each stock
# support_coefs, resist_coefs = fit_trendlines_high_low(ticker_df['High'], ticker_df['Low'], ticker_df['Close'])

In [97]:
# # step 2: calculate trendlines
# x_values = np.arange(len(ticker_df))  # Use the actual data range

# # Calculate the line values using the coefficients
# support_line = support_coefs[0] * x_values + support_coefs[1]
# resistance_line = resist_coefs[0] * x_values + resist_coefs[1]

| ADVANCED CHARTING |
|-|
| Line of Best Fit (Regression Line) |
| Pivot point Xs |
| CLOSE |

In [98]:
# # linear regresion
# last_72_df = ticker_df.iloc[-72:]
# X = np.arange(len(last_72_df)).reshape(-1, 1)
# y = last_72_df['Close'].values.reshape(-1, 1)

# model = LinearRegression().fit(X, y)
# best_fit_last_72 = model.predict(X).flatten()

# # Preparing the regression line plot data
# best_fit_series = pd.Series([np.nan] * len(ticker_df))
# best_fit_series.iloc[-72:] = best_fit_last_72


In [99]:
# # Identify the pivot points within the last 72 periods
# pivot_point_low_index = last_72_df['Close'].idxmin()
# pivot_point_high_index = last_72_df['Close'].idxmax()

In [100]:
# # Initialize series for pivot points with NaN values
# pivot_points_high_series = pd.Series(data=[np.nan] * len(ticker_df), index=ticker_df.index)
# pivot_points_low_series = pd.Series(data=[np.nan] * len(ticker_df), index=ticker_df.index)

# # Set the pivot point values
# pivot_points_high_series[pivot_point_high_index] = ticker_df.loc[pivot_point_high_index, 'Close']
# pivot_points_low_series[pivot_point_low_index] = ticker_df.loc[pivot_point_low_index, 'Close']

| ADVANCED CHARTING |
|-|
| Extending the trend and regression lines to the future |

In [101]:
# # Extend x_values for future predictions
# future_line_periods = 10
# current_periods = len(ticker_df)
# extended_x_values = np.arange(current_periods + future_line_periods)

# # Calculate extended trend lines
# extended_support_line = support_coefs[0] * extended_x_values + support_coefs[1]
# extended_resistance_line = resist_coefs[0] * extended_x_values + resist_coefs[1]

# # Calculate extended line of best fit
# X_future = np.arange(-72, future_line_periods).reshape(-1, 1)  # -72 for last 72 periods and future periods
# extended_best_fit = model.predict(X_future).flatten()

| ADVANCED CHARTING - CHART IT |
|-|
| Candlestick Chart (KEY Levels overlayed) |


In [102]:
# # prepare the plot
# apds = [mpf.make_addplot(support_line, color='green', width=2),
#         mpf.make_addplot(resistance_line, color='red', width=2),
#         mpf.make_addplot(best_fit_series, color='orange', width=2)]        
#         # mpf.make_addplot(pivot_points_high_series, type='scatter', markersize=100, marker='x', color='green'),
#         # mpf.make_addplot(pivot_points_low_series, type='scatter', markersize=100, marker='x', color='red')]

In [103]:
# # look at the df
# ticker_df.tail(10)

# # youre going to have to create some placeholders here.
# # another df?

In [104]:
# #### CHART IT ####
# # Define the period for the x-axis view and future extension
# lookback_periods = 90
# future_periods = 20

# # Calculate the extended index for the lookback period
# lookback_index = ticker_df.index[-lookback_periods]

# # Generate the future index by adding the future periods to the last date
# future_index = pd.date_range(start=ticker_df.index[-1] + pd.Timedelta(days=1),
#                              periods=future_periods, freq='B')

# # Combine the current and future indices
# full_index = ticker_df.index.union(future_index)

# # Set the xlim for plotting
# xlim = (lookback_index, full_index[-1])

# # Plot the candlestick chart with extended x-axis
# mpf.plot(ticker_df,
#          type='candle',
#          style='charles',
#          volume=True,
#          title=f'${ticker} CHART',
#          figratio=(42, 14),
#          xlim=xlim,  # Set the new x-axis limits
#          addplot=apds)  # 'apds' should contain all your additional plots

In [105]:
# ticker_df.tail(10)

In [106]:
# support_line

# END ADVANCED WIP TESTS

| DATA PACKAGE |
|-|
| Trend Lines |


In [107]:
# # Step 3: Calculate the y-values for the line of best fit for the last 72 periods
# best_fit_last_72 = model.predict(X)

# # Create a series for the best fit line, filled with NaN except for the last 72 values
# best_fit_series = pd.Series([np.nan] * len(ticker_df), index=ticker_df.index)
# # best_fit_series = pd.Series(model.predict(X), index=last_72_df.index)
# best_fit_series[-72:] = best_fit_last_72

# # # Step 4: Pivot Points
# # pivot_point_low_index = last_72_df['Close'].idxmin()
# # pivot_point_low_value = last_72_df.loc[pivot_point_low_index, 'Close']

# # pivot_point_high_index = last_72_df['Close'].idxmax()
# # pivot_point_high_value = last_72_df.loc[pivot_point_high_index, 'Close']

# # # Create a series for the pivot points, similar to the best fit line
# # pivot_points_low = pd.Series([np.nan] * len(ticker_df), index=ticker_df.index)
# # pivot_points_low[pivot_point_low_index] = pivot_point_low_value

# # pivot_points_high = pd.Series([np.nan] * len(ticker_df), index=ticker_df.index)
# # pivot_points_high[pivot_point_high_index] = pivot_point_high_value

# # # Step X: chart it
# # last_72_index = ticker_df.index[-90] # 90 periods on the x-axis

# # mpf.plot(ticker_df,
# #          type='candle',
# #          style='charles',
# #          volume=True,
# #          title=f'{ticker} Candlestick Chart: Last 72 Periods with Regression Line and Pivot Point',
# #          figratio=(42, 14),
# #          xlim=(last_72_index, ticker_df.index[-1]),  # Adjusted for the last 72 periods
# #          addplot=apds)

# # mpf.show()


# # # step 5: chart setup
# # apds = [
# #     mpf.make_addplot(best_fit_series, color='orange', width=2),
# #     mpf.make_addplot(pivot_points_low, type='scatter', markersize=200, marker='x', color='red'),
# #     mpf.make_addplot(pivot_points_high, type='scatter', markersize=200, marker='x', color='green')  # Green X for the highest point
# # ]

# # # Step X: chart it
# # last_72_index = ticker_df.index[-90] # 90 periods on the x-axis

# # mpf.plot(ticker_df,
# #          type='candle',
# #          style='charles',
# #          volume=True,
# #          title=f'{ticker} Candlestick Chart: Last 72 Periods with Regression Line and Pivot Point',
# #          figratio=(42, 14),
# #          xlim=(last_72_index, ticker_df.index[-1]),  # Adjusted for the last 72 periods
# #          addplot=apds)

# # mpf.show()

In [108]:
# # step 5: chart setup
# apds = [
#     mpf.make_addplot(best_fit_series, color='orange', width=2),
#     mpf.make_addplot(pivot_points_low, type='scatter', markersize=200, marker='x', color='red'),
#     mpf.make_addplot(pivot_points_high, type='scatter', markersize=200, marker='x', color='green')  # Green X for the highest point
# ]

In [109]:
# # Step X: chart it
# last_72_index = ticker_df.index[-90] # 90 periods on the x-axis

# mpf.plot(ticker_df,
#          type='candle',
#          style='charles',
#          volume=True,
#          title=f'{ticker} Candlestick Chart: Last 72 Periods with Regression Line and Pivot Point',
#          figratio=(42, 14),
#          xlim=(last_72_index, ticker_df.index[-1]),  # Adjusted for the last 72 periods
#          addplot=apds)

# mpf.show()


50DSMA - needed bc everyone has it
standard deviations (2 above)
63DMA - makes at least more sense (63 trading days in a quarter)
2dVWAP
3dVWAP
AVWAP

| Dataframe -> .csv file |
|-|

In [110]:
# # create a csv file with a flat date index (otherwise it'll get lost when saving to csv)
# ticker_csv_df = ticker_df.reset_index()
# # dtype of date
# print(ticker_csv_df['date'].dtype)
# # print index type
# print(ticker_csv_df.index.dtype)

# # save to csv
# ticker_csv_df.to_csv(f'{ticker}_historical_data{today_date_str}', index=False)

# ticker_csv_df.head(2)


In [111]:
# #### add VWAP to df
# chart_historical_df['VWAP'] = (chart_historical_df['volume'] * (chart_historical_df['high'] + chart_historical_df['low'] + chart_historical_df['close']) / 3).cumsum() / chart_historical_df['volume'].cumsum()

# chart_historical_df.head(4)

In [112]:
# """SIMPLE MOVING AVERAGES (SMA)"""
# def calculate_sma(dataframe, period):
#     return dataframe['close'].rolling(window=period).mean()

# chart_historical_df['SMA5'] = calculate_sma(chart_historical_df, 5)
# chart_historical_df['SMA10'] = calculate_sma(chart_historical_df, 10)
# chart_historical_df['SMA15'] = calculate_sma(chart_historical_df, 15)

# chart_historical_df.tail(10)


In [113]:
# """EXPONENTIAL MOVING AVERAGES (EMA)"""
# def calculate_ema(dataframe, period):
#     return dataframe['close'].ewm(span=period, adjust=False).mean()

# # calculate EMAs for the specified periods
# chart_historical_df['EMA5'] = calculate_ema(chart_historical_df, 5)
# chart_historical_df['EMA10'] = calculate_ema(chart_historical_df, 10)
# chart_historical_df['EMA20'] = calculate_ema(chart_historical_df, 20)
# chart_historical_df['EMA50'] = calculate_ema(chart_historical_df, 50)

# chart_historical_df.tail(10)


In [114]:
# """MOVING AVERAGE CONVERGENCE DIVERGENCE (MACD)"""
# # calculate the MACD line and the Signal line
# chart_historical_df['MACD'] = calculate_ema(chart_historical_df, 12) - calculate_ema(chart_historical_df, 26)
# chart_historical_df['MACD_Signal'] = chart_historical_df['MACD'].ewm(span=9, adjust=False).mean()

# # MACD Histogram -> useful for visualizing momentum and potential buy/sell signals
# chart_historical_df['MACD_Histogram'] = chart_historical_df['MACD'] - chart_historical_df['MACD_Signal']

# chart_historical_df.tail(10)

In [115]:
# #### DEV - DATA ORGANIZATION ####
# chart_historical_df.columns

In [116]:
# """BOLLINGER BANDS (BB)"""
# # BB 1 of 2 -> standard bollinger bands (20D CLOSE $)
# period = 20 # SMA periods
# std_dev_multiplier = 2 # standard deviation multiplier

# # line 1: middle band -> simple moving average (SMA)
# chart_historical_df[f'middle_band_SMA{period}'] = chart_historical_df['close'].rolling(window=period).mean()

# # not a line: standard deviation -> used to calculate the upper and lower bands
# chart_historical_df[f'std_dev_SMA{period}'] = chart_historical_df['close'].rolling(window=period).std()

# # lines 2 & 3: upper and lower bands
# chart_historical_df[f'upper_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] + (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)
# chart_historical_df[f'lower_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] - (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)


# # BB 2 of 2 -> typical price bollinger bands (20D TYPICAL $)
# chart_historical_df['typical_price'] = (chart_historical_df['high'] + chart_historical_df['low'] + chart_historical_df['close']) / 3

# # Calculate the SMA of the Typical Price and name it 'middle_band_typical_SMA20'
# chart_historical_df[f'middle_band_typical_SMA{period}'] = chart_historical_df['typical_price'].rolling(window=period).mean()

# # Calculate the standard deviation of the Typical Price
# chart_historical_df[f'std_dev_typical_SMA{period}'] = chart_historical_df['typical_price'].rolling(window=period).std()

# # Calculate the upper and lower bands for the Typical Price and name them accordingly
# chart_historical_df[f'upper_band_typical_SMA{period}'] = chart_historical_df[f'middle_band_typical_SMA{period}'] + (chart_historical_df[f'std_dev_typical_SMA{period}'] * std_dev_multiplier)
# chart_historical_df[f'lower_band_typical_SMA{period}'] = chart_historical_df[f'middle_band_typical_SMA{period}'] - (chart_historical_df[f'std_dev_typical_SMA{period}'] * std_dev_multiplier)

# # view df with new (2) BBs (6 new columns)
# # chart_historical_df.iloc[-1] # last row slice to check the new columns
# chart_historical_df.tail(2)


In [117]:
# chart_historical_df.head(5)

In [118]:
# # drop rows that contain NAs (pulled extra data earlier for this purpose)
# chart_historical_df = chart_historical_df.dropna()

# # view the df
# chart_historical_df.head(5)

In [119]:
# # date column check data type
# chart_historical_df.index

In [120]:
# # IMPORTANT: reset index before saving to Excel (need this shit flat so we don't lose the 'date' index)
# chart_historical_df.reset_index(inplace=True)

In [121]:
# chart_historical_df.index

In [122]:
# chart_historical_df.head(5)

In [123]:
# print(tv_L2_df.columns)

In [124]:
# # takes the original DataFrame and a list of stock symbols,
# # returns a dictionary of DataFrames, one for each stock symbol
# def create_symbol_dataframes(df, symbols):
#     symbol_dfs = {}
#     for symbol in symbols:
#         # Filter the DataFrame for the symbol and store it in a dictionary
#         symbol_dfs[f"{symbol}_chart_df"] = df[df['symbol'] == symbol]
    
#     return symbol_dfs


# symbols = mvp_chart_df['symbol'].unique() 
# symbol_dataframes = create_symbol_dataframes(mvp_chart_df, symbols)

# mtn_df = symbol_dataframes['MTN_chart_df']

# print(symbol_dataframes)



| CALCULATIONS - J. Welles Wilder Jr (technical indicators) |
|-|
| *displayed underneath the chart not overlayed* |
| START |

In [125]:
# """RELATIVE STRENGTH INDEX (RSI)"""
# def calculate_rsi(dataframe, periods=14):
#     close_delta = dataframe['close'].diff()
#     up = close_delta.clip(lower=0)
#     down = -1 * close_delta.clip(upper=0)

#     ma_up = up.rolling(window=periods).mean()
#     ma_down = down.rolling(window=periods).mean()

#     rsi = ma_up / (ma_up + ma_down) * 100
#     return rsi

# # Adding the RSI to your DataFrame
# chart_historical_df['RSI'] = calculate_rsi(chart_historical_df)

# chart_historical_df.tail(10)


In [126]:
# """AVERAGE TRUE RANGE (ATR)"""
# def calculate_atr(dataframe, period=14):
#     high_low = dataframe['high'] - dataframe['low']
#     high_close = (dataframe['high'] - dataframe['close'].shift()).abs()
#     low_close = (dataframe['low'] - dataframe['close'].shift()).abs()

#     # Calculate the True Range (TR) as the maximum of the three measures
#     tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

#     # Calculate the ATR by taking the rolling mean of the TR
#     atr = tr.rolling(window=period).mean()
    
#     return atr

# # Adding the ATR to your DataFrame
# chart_historical_df['ATR'] = calculate_atr(chart_historical_df)

# chart_historical_df.tail(10)

In [127]:
# START HERE
# see about if i should be doing a multi-index and group by if it makes the code more efficient and accurate and all that
# simple data plot soon to see what we are working with and if there are bugs

In [128]:
"""Average Directional Index (ADX)"""

'Average Directional Index (ADX)'

In [129]:
"""PARABOLIC SAR (PSAR)"""

'PARABOLIC SAR (PSAR)'

| J. Welles Wilder Jr (technical indicators) |
|-|
| END |

In [130]:
# LINE LEVELS not full calculation dataframe
# part 1 was calculations append
# part 2 is data points append
# part 3 extra - could be plotting past signals (might integrate ai in here like tensorflow)

| CHARTS |
|-|

In [131]:
# #### xlwings -> dynamic send to excel ####
# # save name for todays returend gap up stocks historical data
# chart_data_file = f'chart_data{today_date_str}.xlsx'

# # reset index before saving to Excel (need
# chart_historical_df.reset_index(inplace=True, drop=True)

# # Save the DataFrame to an Excel file
# chart_historical_df.to_excel(chart_data_file, index=False)

# # Use xlwings to open the file in Excel
# app = xw.App(visible=True)
# book = app.books.open(chart_data_file)

In [132]:
# # AVAILABLE COLS
# chart_historical_df.columns

In [133]:
# def plot_symbol_technicals(chart_historical_df, symbol):
#     # FILTER
#     symbol_df = chart_historical_df[chart_historical_df['symbol'] == symbol].copy()
    
#     # print(symbol_df.head(5))
#     # DATA PREP
#     add2plot = [
#         # mpf.make_addplot(symbol_df['SMA5'], color='#7BAFD4', width=1.0, label='SMA5'), 
#         # mpf.make_addplot(symbol_df['SMA10'], color='#003087', width=1.0, label='SMA10'), 
#         mpf.make_addplot(symbol_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP')
#     ]
    
#     # STYLE & PLOT
#     mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})
#     fig, axes = mpf.plot(symbol_df,
#                          type='candle',
#                          addplot=add2plot,
#                          volume=True,
#                          style=mpf_style,
#                         #  style='yahoo',
#                          figratio=(42, 12),
#                          title=f'${symbol} - Historical Data (4 Months)',
#                          tight_layout=True,
#                          returnfig=True)

#     # watermark
#     fig.text(0.5, 0.5, symbol, fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

#     mpf.show()


In [134]:
# # print a list of the unique stock symbols
# unique_symbols = chart_historical_df['symbol'].unique()
# print(f'Unique stock symbols: {unique_symbols}')

In [135]:
# #### EXECUTE CHARTING ####
# # DEV
# plot_symbol_technicals(chart_historical_df, 'GRND')

# # # PRO
# # for unique in unique_symbols:
# #     plot_symbol_technicals(chart_historical_df, unique)

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [136]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [137]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

In [138]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [139]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [140]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [141]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [142]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [143]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)